In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, concatenate, Lambda, Multiply
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop,SGD,Nadam, Adagrad, Adadelta, Ftrl
from tensorflow.keras.regularizers import l1,l2,l1_l2
from tensorflow.keras.initializers import Constant ,Orthogonal, RandomNormal, VarianceScaling, Ones, Zeros
from tensorflow.keras.constraints import Constraint, UnitNorm
from keras.callbacks import Callback, TerminateOnNaN, ModelCheckpoint
from sksurv.metrics import concordance_index_censored as concordance
import math
import time
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

### Data import

In [2]:
Xdf = pd.read_csv('processedExpData.csv',index_col=0)

In [3]:
ydf = pd.read_csv('yData.csv',index_col=0)

In [4]:
cancer = pd.read_csv('cancerData.csv',index_col=0)

### Model and Nested Crossval Functions

In [5]:
#This is the TFCox class

class TFCox():
    def __init__(self, seed=42,norm=False,optimizer='Adam',l1_ratio=1,lbda=0.0001,
                 max_it=50,learn_rate=0.001,momentum=0.1,stop_if_nan=True,stop_at_value=False, cscore_metric=False,suppress_warnings=True,verbose=0):
        
        self.max_it = max_it
        self.tnan = stop_if_nan
        self.tcscore = stop_at_value
        self.lr=learn_rate
        self.cscore=cscore_metric
        np.random.seed(seed)
        tf.random.set_seed(seed)
        self.op = optimizer
        self.l1r = l1_ratio
        self.lbda=lbda
        self.norm = norm
        self.verbose=verbose
        if suppress_warnings == True:
            import warnings
            warnings.filterwarnings('ignore')
       
        if optimizer == 'Adam':
            self.opt = Adam(learn_rate)
        if optimizer == 'SGD':
            self.opt = SGD(learn_rate)
        if optimizer == 'SGDmomentum':
            self.opt = SGD(learn_rate,momentum=momentum)
        if optimizer == 'RMSprop':
            self.opt = RMSprop(learn_rate)
        if optimizer == 'Ftrl':
            self.opt = Ftrl(learn_rate,l1_regularization_strength=l1_ratio*lbda*100, l2_regularization_strength=(1-l1_ratio)*lbda*100)
            #self.opt = Ftrl(learn_rate,l1_regularization_strength=1, l2_regularization_strength=0)
            
    def coxloss(self, state):        
        def loss(y_true, y_pred):  
                return -K.mean((y_pred - K.log(tf.math.cumsum(K.exp(y_pred),reverse=True,axis=0)+0.0001))*state,axis=0)

        return loss

    def cscore_metric(self, state):
        def loss(y_true,y_pred):
            con = 0
            dis = 0
            for a in range(len(y_pred)):
                for b in range(a+1,len(y_pred)):                                       
                        if (y_pred[a]>y_pred[b])  & (y_pred[a]*state[a]!=0):
                            con+=1
                            
                        elif (y_pred[a]<y_pred[b])  & (y_pred[a]*state[a]!=0):
                            dis+=1
            return     con/(con+dis)
        return loss
 
    
    def fit(self, X,state,time):
        from tensorflow.python.framework.ops import disable_eager_execution
        disable_eager_execution()
        K.clear_session()
       
        
        
        self.time = np.array(time)  
        self.newindex = pd.DataFrame(self.time).sort_values(0).index
        self.X = (pd.DataFrame(np.array(X)).reindex(self.newindex))                      
        self.state = np.array(pd.DataFrame(np.array(state)).reindex(self.newindex))
        self.time  = np.array(pd.DataFrame(np.array(time)).reindex(self.newindex))                       
        inputsx = Input(shape=(self.X.shape[1],)) 
        state = Input(shape=(1,))
        
#         if self.op == 'Ftrl':
#             if self.norm==True:
#                 out = BatchNormalization()(inputsx)
#                 out = Dense(1,activation='linear', use_bias=False)(out)
#             else:
#                 out = Dense(1,activation='linear', use_bias=False)(inputsx)
                
#         else:
        if self.norm==True:
            out = BatchNormalization()(inputsx)
            if self.op!='Ftrl':
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                    kernel_regularizer=l1_l2(self.lbda*self.l1r,self.lbda*(1-self.l1r)),
                   use_bias=False)(out)
            else:
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                   use_bias=False)(out)
        else:
            if self.op!='Ftrl':
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                    kernel_regularizer=l1_l2(self.lbda*self.l1r,self.lbda*(1-self.l1r)),
                   use_bias=False)(inputsx)
            else:
                out = Dense(1,activation='linear',kernel_initializer=Zeros(),
                   use_bias=False)(inputsx)
        model = Model(inputs=[inputsx, state], outputs=out)
        if (self.tcscore != False) or (self.cscore==True) :
            model.compile(optimizer=self.opt ,
                          loss=self.coxloss(state) , metrics=[self.cscore_metric(state)],
                          experimental_run_tf_function=False)
        else:
            model.compile(optimizer=self.opt ,
                          loss=self.coxloss(state) ,
                          experimental_run_tf_function=False)
        
        self.model=model
        if self.verbose==1:
            print(self.model.summary())

        self.loss_history_ = []
        for its in range(self.max_it):
            self.temp_weights = self.model.get_weights()
           
            tr = self.model.train_on_batch([self.X, self.state],np.zeros(self.state.shape))
           
            self.loss_history_.append(tr) 
            
            if self.verbose == 1:
                if (self.tcscore != False) or (self.cscore==True) :
                    print('loss:', self.loss_history_[-1][0],' C-score: ',self.loss_history_[-1][1] )
                else:
                    print('loss:', self.loss_history_[-1] )
            
            if self.tcscore != False:
                if self.loss_history_[-1][1]>=self.tcscore:
                    print('Terminated early because concordance >=' +str(self.tcscore)+ ' as set by stop_at_value flag.')
                    break
            if (self.tcscore != False) or (self.cscore==True) :
                if (math.isnan(self.loss_history_[-1][0]) or math.isinf(self.loss_history_[-1][0])) and self.tnan:
                    self.model.set_weights(self.temp_weights)
                    print('Terminated because weights == nan or inf, reverted to last valid weight set')
                    break
            else:
                if (math.isnan(self.loss_history_[-1]) or math.isinf(self.loss_history_[-1])) and self.tnan:
                    self.model.set_weights(self.temp_weights)
                    print('Terminated because weights == nan or inf, reverted to last valid weight set')
                    break
            
        self.beta_ = self.model.get_weights()[-1]

    def predict(self,X):
        preds = self.model.predict([X,np.zeros(len(X))])

        return preds

In [6]:
#This is a function that runs stratified nested shuffle splits using the TFCox class

def nestedshuffle(lbda=[0.2], l1_r=1,learn_rate=[0.001],max_it=[50],optimizer=['Adam'] ,outloop=5,inloop=5,verbose=1):
    i=0
    np.random.seed(42)
    t0=time.time()
    index_map = pd.DataFrame(range(len(Xdf)),index=Xdf.index)

    outres_train = {}
    outres = {}
    inner_best = {}
    inner_best_train={}
    inner_var = {}
    errors = {}
    preds={}
    weights={}
    timeO = {}
    
       
    
    for out in range(outloop):
        
        t1=time.time()

        totres_train = pd.DataFrame()
        totres = pd.DataFrame()
        totres_var = pd.DataFrame()
        idx=[]
        for cnc in list(cancer):
            for st in [0,1]:
                np.random.seed(out)
                idx.extend(index_map.loc[ np.random.choice(ydf[(ydf['vital_status']==st) & (cancer[cnc]==1)].index, 
                                        size=int(0.8*len(ydf[(ydf['vital_status']==st) & (cancer[cnc]==1)])),replace=False)][0].tolist())


        idx = np.sort(np.ravel(np.array(idx)))

        SS = StandardScaler()
        X_train = Xdf.iloc[idx,:]
        X_train  = pd.DataFrame(SS.fit_transform(X_train),index=X_train.index,columns=X_train.columns)
        X_test = Xdf.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        X_test  = pd.DataFrame(SS.transform(X_test),index=X_test.index,columns=X_test.columns)
        y_train = ydf.iloc[idx,:]
        y_test = ydf.iloc[[x for x in range(len(Xdf)) if x not in idx],:]
        can_train = cancer.iloc[idx,:]
        can_test = cancer.iloc[[x for x in range(len(Xdf)) if x not in idx],:]

        
        for lb in lbda:
            for op in optimizer:
                for lr in learn_rate:
                    for it in max_it:

                        t1=time.time()
                        results = pd.DataFrame()
                        trainresults = pd.DataFrame()
                        index_map2 = pd.DataFrame(range(len(X_train)),index=X_train.index)
                        for b in range(inloop):
                            t2=time.time()
                            K.clear_session()


                            idx2 = []
                            for cnc in list(cancer):
                                for st in [0,1]:
                                    np.random.seed(b)
                                    idx2.extend(index_map2.loc[ np.random.choice(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)].index, 
                                                            size=int(0.75*len(y_train[(y_train['vital_status']==st) & (can_train[cnc]==1)])),replace=False)][0].tolist())


                            idx2 = np.sort(np.ravel(np.array(idx2)))
                            SS = StandardScaler()
                            X_train2 = X_train.iloc[idx2,:]

                            X_train2  = pd.DataFrame(SS.fit_transform(X_train2),index=X_train2.index,columns=X_train2.columns)
                            X_test2 = X_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
                            X_test2  = pd.DataFrame(SS.transform(X_test2),index=X_test2.index,columns=X_test2.columns)
                            y_train2 = y_train.iloc[idx2,:]
                            y_test2 = y_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]
                            can_train2 = can_train.iloc[idx2,:]
                            can_test2 = can_train.iloc[[x for x in range(len(X_train)) if x not in idx2],:]

                            resdict = {}
                            trainresdict = {}
                            for a in list(cancer):

                                Xt2 = X_train2[can_train2[a]==1]
                                Xts2 = X_test2[can_test2[a]==1]




                                K.clear_session()


                                model = TFCox(lbda = lb,optimizer=op,max_it=it,learn_rate=lr,l1_ratio=l1_r,verbose=verbose)


                                model.fit(Xt2,y_train2['vital_status'][can_train2[a]==1],y_train2['time'][can_train2[a]==1])


                                train_pred =  model.predict(Xt2).flatten()
                                pred = model.predict(Xts2).flatten()


                                resdict[a] = pred
                                trainresdict[a] = train_pred

                            res = pd.DataFrame()
                            trainres = pd.DataFrame()

                            for a in list(cancer):
                              #  print(trainresdict[a].shape)
                               # print(y_train2['vital_status'][can_train2[a]==1].astype('bool').shape)
                                try:

                                    trainres.loc[a,0]= concordance(y_train2['vital_status'][can_train2[a]==1].astype('bool'),y_train2['time'][can_train2[a]==1],trainresdict[a])[0]
                                except:

                                    trainres.loc[a,0] = 'fail'

                                try:
                                    res.loc[a,0]= concordance(y_test2['vital_status'][can_test2[a]==1].astype('bool'),y_test2['time'][can_test2[a]==1],resdict[a])[0]

                                except:
                                    res.loc[a,0] = 'fail'


                            print(i,'/',len(lbda) *outloop*(inloop+1) )
                            print((time.time()-t0)/60,(time.time()-t1)/60,(time.time()-t2)/60)
                            results[str(b)] = res[0]
                            trainresults[str(b)] = trainres[0]
                            i+=1

                        totres_var[frozenset({'lambda':lb,'iter':it,'rate':lr,'opt':op}.items())] = results.var(1)
                        totres[frozenset({'lambda':lb,'iter':it,'rate':lr,'opt':op}.items())] = results.mean(1)
                        totres_train[frozenset({'lambda':lb,'iter':it,'rate':lr,'opt':op}.items())] = trainresults.mean(1)


        inner_best[out] = totres
        inner_best_train[out] = totres_train
        inner_var[out] = totres_var
        outres[out] = pd.DataFrame()
        outres_train[out] = pd.DataFrame()
        timeO[out] = pd.DataFrame()
        
        
        
        for prms in list(inner_best[out]):
            weights[str(out)+str(prms)] = pd.DataFrame()
            K.clear_session()
            params = dict(prms)


            resdict = {}
            trainresdict = {}
            for a in list(cancer):
                
                Xt = X_train[can_train[a]==1]
                Xts = X_test[can_test[a]==1]
                
                
                t5=time.time()
             


                K.clear_session()

                model = TFCox(lbda = params['lambda'],optimizer=params['opt'],max_it=params['iter'],learn_rate=params['rate'],l1_ratio=l1_r,verbose=verbose)


                model.fit(Xt,y_train['vital_status'][can_train[a]==1],y_train['time'][can_train[a]==1])


                train_pred =  model.predict(Xt).flatten()
                pred = model.predict(Xts).flatten()

                weights[str(out)+str(prms)][a]=model.model.get_weights()
                resdict[a] = pred
                trainresdict[a] = train_pred

                #timeO[out].loc[a,frozenset(params.items())] = time.time()-t5
                
            res = pd.DataFrame()
            trainres = pd.DataFrame()
            for a in list(cancer):
                try:

                    trainres.loc[a,0]= concordance(y_train['vital_status'][can_train[a]==1].astype('bool'),y_train['time'][can_train[a]==1],trainresdict[a])[0]
                except:

                    trainres.loc[a,0] = 'fail'
                try:
                    res.loc[a,0]= concordance(y_test['vital_status'][can_test[a]==1].astype('bool'),y_test['time'][can_test[a]==1],resdict[a])[0]

                except:
                    res.loc[a,0] = 'fail'

            print(i,'/',len(lbda) *outloop*(inloop+1) )
            print((time.time()-t0)/60,(time.time()-t1)/60,(time.time()-t2)/60)
            i+=1
            outres[out][frozenset(params.items())] = res[0]
            outres_train[out][frozenset(params.items())] = trainres[0]
          
    return outres , outres_train,inner_best,inner_best_train,inner_var,weights#,timeO

### Running the model

In [5]:
# This is the geometric range of lambdas
vals = [float(np.format_float_positional(1 * 0.75**i, precision=2, unique=False, fractional=False, trim='k')) for i in range(33)]+[0]


In [12]:
# Nested shuffle splits with LASSO Regularization and FTRL optimizer
lassoFtO1,lassoFtO2,lassoFtI1,lassoFtI2,lassoFtvar,lassoFtwt= nestedshuffle(lbda=vals,l1_r=1,optimizer=['Ftrl'],learn_rate=[0.001],max_it=[150] ,outloop=5, inloop=5,verbose=0)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
0 / 12
1.8189251939455668 1.768534036477407 1.768534036477407
1 / 12
3.5928327401479083 1.7738909244537353 1.773874298731486
2 / 12
5.366705858707428 1.7738564968109132 1.7738398750623068
3 / 12
7.149780937035879 1.7830418348312378 1.7830418348312378
4 / 12
8.927265135447184 1.777450954914093 1.7774344404538474
5 / 12
10.709524329503378 1.7822258631388346 1.7822258631388346
6 / 12
12.969949805736542 4.042651339371999 4.042651339371999
7 / 12
15.232163751125336 6.304865284760793 6.304865284760793
8 / 12
17.492656354109446 8.565357887744904 8.565357887744904
9 / 12
19.765010146299996 10.837711679935456 10.837711679935456
10 / 12
22.033997174104055 13.106698707739513 13.106698707739513
11 / 12
24.310832754770914 15.383534288406372 15.383534288406372


In [399]:
# Nested shuffle splits with elnet Regularization and FTRL optimizer
elnetFtO1,elnetFtO2,elnetFtI1,elnetFtI2,elnetFtvar,elnetFtwt= nestedshuffle(lbda=vals,l1_r=0.5,optimizer=['Ftrl'],learn_rate=[0.001],max_it=[150] ,outloop=5, inloop=5,verbose=0)

0 / 12
1.8375487645467123 1.7847579876581827 1.7847579876581827
1 / 12
3.6534220496813457 1.8158399979273478 1.8158233880996704
2 / 12
5.474298103650411 1.820859440167745 1.8208428064982096
3 / 12
7.2945594390233355 1.820228091875712 1.820228091875712
4 / 12
9.111563527584076 1.8169874509175619 1.8169708728790284
5 / 12
10.926359740893046 1.8147629698117573 1.8147629698117573
6 / 12
13.220516804854075 4.108920033772787 4.108920033772787
7 / 12
15.537620961666107 6.426024190584818 6.426024190584818
8 / 12
17.855932033061983 8.744335261980693 8.744335261980693
9 / 12
20.178848481178285 11.067251710096995 11.067251710096995
10 / 12
22.49932949145635 13.38773272037506 13.38773272037506
11 / 12
24.81294479370117 15.701348022619884 15.701348022619884


In [201]:
# Nested shuffle splits with LASSO Regularization and Adam optimizer
lassoAdO1,lassoAdO2,lassoAdI1,lassoAdI2,lassoAdvar,lassoAdwt= nestedshuffle(lbda=vals,l1_r=1,optimizer=['Adam'],learn_rate=[0.001],max_it=[50] ,outloop=5, inloop=5,verbose=0)

0 / 1020
0.6125319480895997 0.5617837389310201 0.5617837389310201
1 / 1020
1.1745158712069192 1.1237676620483399 0.5619673132896423
2 / 1020
1.7374031941095989 1.6866549849510193 0.5628873229026794
3 / 1020
2.304403209686279 2.2536550005277 0.5669833819071451
4 / 1020
2.866819405555725 2.8160711963971456 0.5623995343844096
5 / 1020
3.4311679641405743 0.5642986973126729 0.5642986973126729
6 / 1020
3.994910399119059 1.128041132291158 0.5637262662251791
7 / 1020
4.561916077136994 1.695046810309092 0.5669890403747558
8 / 1020
5.125200589497884 2.258331322669983 0.5632678826649984
9 / 1020
5.688829139868418 2.8219598730405173 0.5636285503705343
10 / 1020
6.252489654223124 0.5636272668838501 0.5636272668838501
11 / 1020
6.818343404928843 1.129481017589569 0.5658371408780416
12 / 1020
7.383529078960419 1.6946666916211446 0.565169095993042
13 / 1020
7.946939440568288 2.258077053229014 0.5633771061897278
14 / 1020
8.511447664101919 2.8225852767626445 0.5645082235336304
15 / 1020
9.0780499339103

125 / 1020
71.0287062883377 0.5616073846817017 0.5615907311439514
126 / 1020
71.59027775128682 1.1231788476308187 0.5615548412005107
127 / 1020
72.15231676101685 1.6852178573608398 0.5620223720868428
128 / 1020
72.71580641269684 2.2487075090408326 0.5634896516799927
129 / 1020
73.27742763360341 2.810328729947408 0.5616044878959656
130 / 1020
73.83905706803004 0.5615961790084839 0.5615961790084839
131 / 1020
74.40146429141363 1.1240034023920695 0.5624072233835856
132 / 1020
74.96520850658416 1.6877476175626118 0.563727597395579
133 / 1020
75.52655115127564 2.249090262254079 0.5613259871800741
134 / 1020
76.08858054478964 2.8111196557680764 0.562012775739034
135 / 1020
76.64972967704138 0.5611158967018127 0.5611158967018127
136 / 1020
77.21307802597681 1.124464245637258 0.5633317788441976
137 / 1020
77.7753807425499 1.6867669622103374 0.5622860948244731
138 / 1020
78.33813454310099 2.249520762761434 0.5627538005510966
139 / 1020
78.89967823425928 2.8110644539197285 0.5615436911582947
140

248 / 1020
143.4579000035922 2.807346244653066 0.5605918208758036
249 / 1020
144.01786452929178 0.5599312464396159 0.5599146604537963
250 / 1020
144.58018499215444 1.1222517093022664 0.562303900718689
251 / 1020
145.13978428840636 1.6818510055541993 0.5595826784769694
252 / 1020
145.69928002357483 2.241346740722656 0.5594789028167725
253 / 1020
146.26060706774393 2.8026737848917644 0.5613109270731608
254 / 1020
146.82241611878078 0.5617591857910156 0.5617591857910156
255 / 1020
147.38275617758433 1.1220992445945739 0.560323445002238
256 / 1020
147.9438050587972 1.6831481258074443 0.5610322594642639
257 / 1020
148.50408841371535 2.243431480725606 0.5602667212486268
258 / 1020
149.0660513361295 2.80539440313975 0.5619462728500366
259 / 1020
149.625532301267 0.559447721640269 0.5594310919443767
260 / 1020
150.18490131696066 1.1188167373339335 0.5593690156936646
261 / 1020
150.74606481790542 1.6799802382787068 0.5611468911170959
262 / 1020
151.3099146882693 2.243830108642578 0.563833244641

371 / 1020
212.48502420584362 1.6614160537719727 0.5534717321395874
372 / 1020
213.04158694346745 2.217978791395823 0.5565627376238506
373 / 1020
213.59583037296932 2.7722222208976746 0.554226811726888
374 / 1020
214.2526692390442 3.4290610869725544 1.2110656778017679
375 / 1020
214.9103940407435 4.086785888671875 1.8687904795010886
376 / 1020
215.57090145746866 4.747293305397034 2.529297896226247
377 / 1020
216.22968519131342 5.406077039241791 3.188081630071004
378 / 1020
216.88746892213823 6.063860770066579 3.8458653608957927
379 / 1020
217.54514283736546 6.721534685293833 4.503539276123047
380 / 1020
218.20575037797292 7.382142225901286 5.164146816730499
381 / 1020
218.864366265138 8.040758113066355 5.8227627038955685
382 / 1020
219.52270099322 8.699092841148376 6.48109743197759
383 / 1020
220.17981694936753 9.356208797295889 7.138213388125101
384 / 1020
220.84010243813196 10.016494286060333 7.798498876889547
385 / 1020
221.49832247893016 10.67471432685852 8.456718917687734
386 / 10

494 / 1020
284.86031072537105 1.1229301373163858 0.5604967435201009
495 / 1020
285.42133019367856 1.683949605623881 0.5610028465588888
496 / 1020
285.98251400788627 2.245133419831594 0.5611671884854634
497 / 1020
286.54495743910473 2.807576851050059 0.5624268492062886
498 / 1020
287.1053984562556 0.5604077577590942 0.5604077577590942
499 / 1020
287.66605790456134 1.12106720606486 0.5606427192687988
500 / 1020
288.22715882062914 1.6821681221326192 0.5611009160677592
501 / 1020
288.7900994459788 2.2451087474822997 0.5629406253496806
502 / 1020
289.3510704636574 2.8060797651608786 0.5609548370043437
503 / 1020
289.91222185293833 0.5611181457837423 0.5611010352770488
504 / 1020
290.4724387486776 1.1213350415229797 0.5602001388867696
505 / 1020
291.035464600722 1.6843608935674033 0.5630090912183126
506 / 1020
291.59562904834746 2.2445253411928814 0.5601644476254781
507 / 1020
292.1554161429405 2.8043124357859295 0.5597706079483032
508 / 1020
292.71566795110704 0.560218604405721 0.5602019906

617 / 1020
357.3701067527135 0.5633854786554973 0.5633688529332479
618 / 1020
357.9323794722557 1.1256581981976828 0.56225612560908
619 / 1020
358.4957262158394 1.689004941781362 0.5633301377296448
620 / 1020
359.060970556736 2.2542492826779683 0.5652275244394939
621 / 1020
359.624319088459 2.8175978144009908 0.5633318305015564
622 / 1020
360.1868586699168 0.5625229636828105 0.5625063419342041
623 / 1020
360.74970531463623 1.1253696084022522 0.5628304998079936
624 / 1020
361.3143603642782 1.6900246580441793 0.564638880888621
625 / 1020
361.87666997114815 2.252334264914195 0.5622929533322653
626 / 1020
362.4401267608007 2.8157910545667013 0.563440187772115
627 / 1020
363.0024664839109 0.5623065789540609 0.5623065789540609
628 / 1020
363.5664852221807 1.1263253172238668 0.5640021443367005
629 / 1020
364.12862980763117 1.688469902674357 0.5621445854504903
630 / 1020
364.6900375088056 2.249877603848775 0.5613910714785258
631 / 1020
365.2517297506332 2.811569845676422 0.5616756240526836
632

740 / 1020
426.4483135263125 2.243787169456482 0.5602017045021057
741 / 1020
427.0107728322347 2.8062464753786722 0.5624426205952963
742 / 1020
427.5709962407748 0.5601900259653727 0.5601900259653727
743 / 1020
428.13273940881095 1.1219331940015158 0.5617264906565348
744 / 1020
428.6939579208692 1.6831517060597738 0.561218512058258
745 / 1020
429.2576123515765 2.2468061367670695 0.5636378129323324
746 / 1020
429.8187667489052 2.807960534095764 0.5611377954483032
747 / 1020
430.3793002406756 0.5605168183644612 0.5605002403259277
748 / 1020
430.9401741782824 1.1213907559712728 0.5608573198318482
749 / 1020
431.50359559059143 1.6848121682802837 0.5634214123090108
750 / 1020
432.06495314041774 2.2461697181065876 0.5613409201304118
751 / 1020
432.6258156736692 2.807032251358032 0.5608458956082661
752 / 1020
433.1871965368589 0.5613476037979126 0.5613314429918925
753 / 1020
433.7504807472229 1.1246318141619365 0.5632842103640239
754 / 1020
434.31100554466246 1.6851566116015115 0.560524797439

863 / 1020
498.78634689251584 1.681271779537201 0.5596888343493144
864 / 1020
499.3464480837186 2.2413729707400005 0.560084593296051
865 / 1020
499.9088811596235 2.8038060466448465 0.5624330759048461
866 / 1020
500.46892786423365 0.5600133856137594 0.5599969188372295
867 / 1020
501.02950332959495 1.1205888509750366 0.5605588555335999
868 / 1020
501.5892164707184 1.6803019920984903 0.5596965233484904
869 / 1020
502.1516188899676 2.2427044113477073 0.5624024192492167
870 / 1020
502.7112129688263 2.8022984902064008 0.5595772345860799
871 / 1020
503.27109038829803 0.559844175974528 0.559844175974528
872 / 1020
503.831106921037 1.1198607087135315 0.559999938805898
873 / 1020
504.3943870345751 1.6831408222516377 0.5632639408111573
874 / 1020
504.95504450003307 2.243798287709554 0.5606412927309672
875 / 1020
505.5150705218315 2.803824309508006 0.5600095272064209
876 / 1020
506.0751658519109 0.5600621740023295 0.5600621740023295
877 / 1020
506.63775709072746 1.1226534128189087 0.56259123881657

987 / 1020
568.5031988302867 4.089341123898824 1.8714783350626627
988 / 1020
569.1609060049057 4.747048298517863 2.5291855096817017
989 / 1020
569.818235139052 5.404377432664235 3.1865146438280743
990 / 1020
570.4756306012472 6.061772894859314 3.8439101060231526
991 / 1020
571.1359157601993 6.722058053811391 4.5041952649752295
992 / 1020
571.7939352909724 7.380077584584554 5.162214795748393
993 / 1020
572.451228471597 8.037370765209198 5.819507976373036
994 / 1020
573.1089106241863 8.69505291779836 6.4771901289621985
995 / 1020
573.7700389107068 9.356181204319 7.138318415482839
996 / 1020
574.4269232352575 10.013065528869628 7.795202740033468
997 / 1020
575.0848951419194 10.671037435531616 8.453174646695455
998 / 1020
575.7422218243281 11.328364117940266 9.110501329104105
999 / 1020
576.4023300290107 11.988472322622934 9.770609533786773
1000 / 1020
577.0602747797966 12.646417073408763 10.428554284572602
1001 / 1020
577.7177609801292 13.303903273741405 11.086040484905244
1002 / 1020
578

In [271]:
# Nested shuffle splits with elnet Regularization and Adam optimizer
elnetAdO1,elnetAdO2,elnetAdI1,elnetAdI2,elnetAdvar,elnetAdwt= nestedshuffle(lbda=vals,l1_r=0.5,optimizer=['Adam'],learn_rate=[0.001],max_it=[50] ,outloop=5, inloop=5,verbose=0)

0 / 1020
0.6187575221061706 0.5685931245485941 0.5685765226682027
1 / 1020
1.1867714842160544 1.1366070866584779 0.5679972290992736
2 / 1020
1.7594648122787475 1.709300414721171 0.5726933280626932
3 / 1020
2.327352527777354 2.2771881302197774 0.567871622244517
4 / 1020
2.905384186903636 2.855219789346059 0.5780154863993326
5 / 1020
3.4787875056266784 0.5733535011609395 0.5733535011609395
6 / 1020
4.0536114494005835 1.148177444934845 0.5748239437739054
7 / 1020
4.628671912352244 1.7232379078865052 0.5750604629516601
8 / 1020
5.201800723870595 2.2963667194048565 0.5731126387914022
9 / 1020
5.77222785949707 2.8667938550313314 0.5704271356264751
10 / 1020
6.348656932512919 0.5763792157173157 0.5763792157173157
11 / 1020
6.920096298058827 1.1478185812632242 0.5714227398236592
12 / 1020
7.492555864651998 1.7202781478563944 0.5724429408709208
13 / 1020
8.06866428454717 2.2963865677515667 0.5760922431945801
14 / 1020
8.643265859285991 2.870988142490387 0.5745849251747132
15 / 1020
9.2217204173

125 / 1020
72.05076038440069 0.5682933966318766 0.5682767788569133
126 / 1020
72.62227752606074 1.1398105382919312 0.5715171416600545
127 / 1020
73.18989527225494 1.7074282844861348 0.5676007906595866
128 / 1020
73.75787889957428 2.2754119118054708 0.5679836273193359
129 / 1020
74.32542833487193 2.842961347103119 0.5675320863723755
130 / 1020
74.89619215329488 0.5707305630048116 0.5707305630048116
131 / 1020
75.46453754107158 1.1390759507815043 0.5683286587397257
132 / 1020
76.03281476497651 1.7073531746864319 0.5682772239049275
133 / 1020
76.60085884332656 2.275397253036499 0.5680274566014608
134 / 1020
77.17146174112956 2.8460001508394877 0.5706028978029887
135 / 1020
77.73956112066905 0.5680661360422771 0.5680661360422771
136 / 1020
78.3077153046926 1.1362203200658163 0.5681373318036397
137 / 1020
78.87630583445231 1.7048108498255412 0.5685905297597249
138 / 1020
79.44605092604955 2.2745559414227805 0.5697284738222758
139 / 1020
80.01431653499603 2.842821550369263 0.5682656089464824

248 / 1020
145.58822374343873 2.870540527502696 0.5728198289871216
249 / 1020
146.16153424978256 0.5732772588729859 0.5732772588729859
250 / 1020
146.73470167716343 1.1464446862538655 0.5731508096059164
251 / 1020
147.3138974626859 1.7256404717763265 0.5791786869366964
252 / 1020
147.88928749163946 2.301030500729879 0.5753900289535523
253 / 1020
148.46320760647455 2.874950615564982 0.5739036122957866
254 / 1020
149.03770702282588 0.5744662086168925 0.5744494239489237
255 / 1020
149.61147365967432 1.148232845465342 0.5737500150998434
256 / 1020
150.18091208934783 1.717671275138855 0.5694218079249064
257 / 1020
150.7510383963585 2.287797582149506 0.5701095938682557
258 / 1020
151.32033485968907 2.857094045480092 0.5692964633305867
259 / 1020
151.89322491486868 0.5728568156560262 0.5728402336438497
260 / 1020
152.46381973425548 1.1434516350428263 0.5705610911051432
261 / 1020
153.03380906184515 1.7134409626324971 0.5699731826782226
262 / 1020
153.60457962354025 2.284211524327596 0.5707537

370 / 1020
215.23841469685237 1.1167427221934 0.5580607295036316
371 / 1020
215.797197107474 1.6755251328150431 0.5587655663490295
372 / 1020
216.3582961877187 2.236624213059743 0.5610990802447001
373 / 1020
216.9169707973798 2.7952988227208455 0.5586584568023681
374 / 1020
217.58823545773825 3.4665634830792746 1.2299231171607972
375 / 1020
218.2576767762502 4.136004801591238 1.89936443567276
376 / 1020
218.9301130135854 4.808441038926443 2.571800673007965
377 / 1020
219.59946126143137 5.47778928677241 3.2411489208539326
378 / 1020
220.26859066088994 6.146918686230977 3.9102783203125
379 / 1020
220.93750535647075 6.815833381811777 4.5791930158933
380 / 1020
221.6098226070404 7.488150632381439 5.251510266462962
381 / 1020
222.27784607807794 8.156174103418985 5.919533737500509
382 / 1020
222.94613989988963 8.824467925230662 6.587827559312185
383 / 1020
223.6152803937594 9.493608419100443 7.256968053181966
384 / 1020
224.28601092100143 10.164338946342468 7.927698580423991
385 / 1020
224.9

493 / 1020
288.9651132186254 0.5730498949686686 0.5730498949686686
494 / 1020
289.5355780204137 1.1435146967569987 0.57046480178833
495 / 1020
290.10608412822086 1.714020804564158 0.5705061078071594
496 / 1020
290.6761551856995 2.284091862042745 0.5700710574785869
497 / 1020
291.24888024727505 2.8568169236183167 0.5727250615755717
498 / 1020
291.8195583144824 0.5706448117891948 0.5706448117891948
499 / 1020
292.39068252245585 1.1417690197626749 0.5711080352465312
500 / 1020
292.96065871715547 1.7117452144622802 0.5699595928192138
501 / 1020
293.53395682175955 2.2850433190663657 0.5732813755671183
502 / 1020
294.1046198924383 2.8557063897450763 0.5706461628278097
503 / 1020
294.6748998324076 0.5702466607093811 0.5702466607093811
504 / 1020
295.2460645039876 1.1414113322893777 0.5711480577786764
505 / 1020
295.8184576670329 1.7138044953346252 0.572376533349355
506 / 1020
296.3881844679515 2.283531296253204 0.5697101593017578
507 / 1020
296.9586750149727 2.854021843274434 0.57047392527262

616 / 1020
362.4622497280439 2.8460745414098105 0.569178573290507
617 / 1020
363.03066890239717 0.5683856288592021 0.5683693051338196
618 / 1020
363.6033403992653 1.1410571257273356 0.5726546843846639
619 / 1020
364.17223560412725 1.7099523305892945 0.5688790321350098
620 / 1020
364.7421873609225 2.279904087384542 0.5699517567952473
621 / 1020
365.3124803582827 2.8501970847447713 0.5702929973602295
622 / 1020
365.8853361487389 0.5728226184844971 0.5728226184844971
623 / 1020
366.4547608176867 1.1422472874323526 0.5694080630938212
624 / 1020
367.0246142546336 1.7121007243792217 0.5698366562525431
625 / 1020
367.5931707024574 2.280657172203064 0.5685397823651631
626 / 1020
368.165810751915 2.853297221660614 0.5726234197616578
627 / 1020
368.73658610184987 0.5707587202390035 0.570742114384969
628 / 1020
369.30659810304644 1.140770721435547 0.5700120011965434
629 / 1020
369.8747360547384 1.7089086731274923 0.5681379516919454
630 / 1020
370.44598360061644 2.2801562190055846 0.57123091618220

739 / 1020
432.71265881061555 1.7328688104947407 0.5755398710568745
740 / 1020
433.2890149911245 2.3092249910036724 0.5763395468393961
741 / 1020
433.8668851296107 2.8870951294898988 0.5778701384862264
742 / 1020
434.44688733021417 0.5799855788548788 0.5799689571062724
743 / 1020
435.02353893915813 1.156637187798818 0.5766349911689759
744 / 1020
435.60102235476177 1.7341206034024557 0.5774834156036377
745 / 1020
436.1767874320348 2.3098856806755066 0.5757484912872315
746 / 1020
436.7564063668251 2.8895046154658 0.579602305094401
747 / 1020
437.3326767762502 0.576237165927887 0.5762210011482238
748 / 1020
437.90849221547444 1.15205260515213 0.5757988174756368
749 / 1020
438.48422955274583 1.7277899424235026 0.5757206598917644
750 / 1020
439.06473881403605 2.308299203713735 0.5804925799369812
751 / 1020
439.64026000897087 2.88382039864858 0.575521194934845
752 / 1020
440.21588939030966 0.5755961298942566 0.5755961298942566
753 / 1020
440.7916681408882 1.151374880472819 0.575762128829956


862 / 1020
506.87111369371416 1.1393628438313803 0.5697157343228658
863 / 1020
507.4425350348155 1.7107841849327088 0.5714213411013286
864 / 1020
508.01849200725553 2.2867411573727927 0.5759569724400838
865 / 1020
508.5907540082932 2.85900315841039 0.5722453991572062
866 / 1020
509.1671270251274 0.5763397932052612 0.5763397932052612
867 / 1020
509.754239988327 1.1634527564048767 0.5870961983998616
868 / 1020
510.3328423817953 1.7420551498730978 0.5786023934682211
869 / 1020
510.9032117327054 2.3124245007832847 0.5703527649243673
870 / 1020
511.4762213349342 2.885434103012085 0.5729929367701213
871 / 1020
512.050791311264 0.5745367328325908 0.5745367328325908
872 / 1020
512.625496629874 1.1492420514424642 0.5746891458829244
873 / 1020
513.1992986996969 1.7230441212654113 0.5737853368123372
874 / 1020
513.7690111955006 2.2927566170692444 0.569712495803833
875 / 1020
514.3373024344444 2.86104785601298 0.5682746171951294
876 / 1020
514.9175528327625 0.5802337964375813 0.5802171309789022
87

986 / 1020
577.6348410367966 3.4483707904815675 1.2211225231488545
987 / 1020
578.2992282787959 4.112758032480875 1.885509765148163
988 / 1020
578.9639572223027 4.7774869759877525 2.5502387086550393
989 / 1020
579.6295416037242 5.443071357409159 3.2158230900764466
990 / 1020
580.2986567378044 6.11218649148941 3.8849382241566977
991 / 1020
580.9642903923989 6.777820146083831 4.550571878751119
992 / 1020
581.6307244976362 7.444254251321157 5.217005983988444
993 / 1020
582.2960912307103 8.109620984395345 5.8823727170626325
994 / 1020
582.9652287483216 8.77875850200653 6.551510234673818
995 / 1020
583.6299737532934 9.443503506978352 7.21625523964564
996 / 1020
584.2948575456937 10.108387299378712 7.881139032046
997 / 1020
584.9604630271593 10.77399278084437 8.546744513511658
998 / 1020
585.6295405189196 11.443070272604624 9.215822005271912
999 / 1020
586.2952555298805 12.10878528356552 9.881537016232809
1000 / 1020
586.9608957489331 12.774425502618154 10.54717723528544
1001 / 1020
587.6265

# Finding the mean concordance from the nested crossval

In [70]:
lassoresFt = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(lassoFtI1[b].index):
        
        
        lassoresFt.loc[a,b] =  lassoFtO1[b][lassoFtI1[b].idxmax(1)[a]][a]

In [71]:
elnetresFt = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(elnetFtI1[b].index):
        
        
        elnetresFt.loc[a,b] =  elnetFtO1[b][elnetFtI1[b].idxmax(1)[a]][a]

In [72]:
lassoresAd = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(lassoAdI1[b].index):
        
        
        lassoresAd.loc[a,b] =  lassoAdO1[b][lassoAdI1[b].idxmax(1)[a]][a]

In [73]:
elnetresAd = pd.DataFrame()
numloops=5
for b in range(numloops):
    for a in list(elnetAdI1[b].index):
        
        
        elnetresAd.loc[a,b] =  elnetAdO1[b][elnetAdI1[b].idxmax(1)[a]][a]

In [85]:
lassoresall = pd.DataFrame([lassoresFt.mean(1),lassoresAd.mean(1)]).transpose()
lassoresall.columns = ['FTRL','Adam']

In [86]:
lassoresall['Cancer'] = list(lassoresall.index)

In [87]:
#Display the lasso results
lassoresall

In [91]:
elnetresall = pd.DataFrame([elnetresFt.mean(1),elnetresAd.mean(1)]).transpose()
elnetresall.columns = ['FTRL','Adam']

In [92]:
elnetresall['Cancer'] = list(elnetresall.index)

In [93]:
#Display the elastic net results
elnetresall